In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === Paths ===
base_dir = "/content/drive/MyDrive/Info_Project/Defect_Detection/DataSets/Data.Splitting/After_Melting_Defect_Detection"
test_img_dir = os.path.join(base_dir, "test", "Img.After.Melting")
test_mask_dirs = [os.path.join(base_dir, "(unet)predicted_binary_masks", f"Defect_Class{i}") for i in [0, 5, 8, 9, 10, 11]]

ground_truth_dir = os.path.join(base_dir, "predicted")
os.makedirs(ground_truth_dir, exist_ok=True)

In [ ]:
# === Color map (index 0-5 corresponds to defect classes 0,5,8,9,10,11) ===
colors = [
    (128, 128, 128),       # Class 0 (no defect)
    (255, 0, 0),     # Class 1 (Defect_Class5)
    (0, 255, 0),     # Class 2 (Defect_Class8)
    (0, 0, 255),     # Class 3 (Defect_Class9)
    (255, 255, 0),   # Class 4 (Defect_Class10)
    (255, 0, 255),   # Class 5 (Defect_Class11)
]


In [ ]:
# === File names (assuming same filenames across all mask folders) ===
file_names = sorted(os.listdir(test_img_dir))


In [ ]:
# === Generate color-coded masks ===
for file_name in tqdm(file_names, desc="Creating GT color masks"):
    class_mask = None

    for class_idx, mask_dir in enumerate(test_mask_dirs):
        mask_path = os.path.join(mask_dir, file_name)
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is not None:
                if class_mask is None:
                    H, W = mask.shape
                    class_mask = np.zeros((H, W), dtype=np.uint8)
                elif mask.shape != class_mask.shape:
                    mask = cv2.resize(mask, (class_mask.shape[1], class_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

                class_mask[mask > 0] = class_idx

    # Skip if no valid masks found
    if class_mask is None:
        continue

    # Create RGB image
    color_mask = np.zeros((class_mask.shape[0], class_mask.shape[1], 3), dtype=np.uint8)
    for c in range(6):
        color_mask[class_mask == c] = colors[c]

    # Save image
    save_path = os.path.join(ground_truth_dir, f"{file_name}")
    cv2.imwrite(save_path, color_mask)

print("✅ All ground truth masks saved in:", ground_truth_dir)

Creating GT color masks: 100%|██████████| 537/537 [11:51<00:00,  1.32s/it]

✅ All ground truth masks saved in: /content/drive/MyDrive/Info_Project/Defect_Detection/DataSets/Data.Splitting/After_Melting_Defect_Detection/predicted
